<p style="text-align:center; font-size:60px; font-weight:bold;">Sakila</p>


<h2 style="font-size:40px; font-weight:bold;">1 : Python Librairies</h2>

In [1]:
# This section imports the libraries we will use for SQLite queries in Python : 

import sqlite3
import pandas as pd
from IPython.display import display

---------------------------------------------------------------------------------------------------------------------------------------------------------

<h2 style="font-size:40px; font-weight:bold;">2 : Opening the tables from Sakila</h2>

In [2]:
# Connection
conn = sqlite3.connect("sakila.db")
conn.text_factory = lambda b: b.decode("latin-1")

In [3]:
# Create a cursor to run SQL commands
cursor = conn.cursor()

In [4]:
# Liste des tables :
tables = "SELECT name FROM sqlite_master WHERE type='table';"
df_tables = pd.read_sql_query(tables, conn)
list_tables = list(df_tables["name"])
df_tables.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
name,actor,country,city,address,language,category,customer,film,film_actor,film_category,film_text,inventory,staff,store,payment,rental


---------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Définition de lecture de requêtes SQL en python :

def requests_sql(requete, conn):
    """
    Exécute une requête SQL et retourne un DataFrame pandas.
    """
    return pd.read_sql(requete, conn)

In [6]:
# Visualisation simplifiée de toutes les tables :

for name_table in list_tables :
    SQL = pd.read_sql(f"SELECT * FROM '{name_table}'", conn).head(1)
    display(name_table, SQL)

'actor'

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2020-02-15 06:59:28


'country'

,country_id,country,last_update
0,1,Afghanistan,2020-02-15 06:59:27


'city'

...

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2020-02-15 06:59:28


'payment'

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76.0,2.99,2005-05-25 11:30:37.000,2020-02-15 06:59:47


'rental'

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30.000,367,130,2005-05-26 22:04:30.000,1,2020-02-15 06:59:36


<h2 style="font-size:40px; font-weight:bold;">3 : Etude de cas</h2>

In [20]:
# Cas N°1 : Clients les plus fidèles
# But : Retourne les 20 clients avec le plus grand nombre de locations, avec leur nombre total de locations, triés du plus grand au plus petit.

query = '''
SELECT
    customer.first_name || ' ' || customer.last_name AS NomClient,
    COUNT(rental.rental_id) AS NombreLocation,
    RANK() OVER(ORDER BY COUNT(rental.rental_id) DESC) AS Ranking
FROM customer
JOIN rental 
    ON customer.customer_id = rental.customer_id
GROUP BY customer.first_name, customer.last_name
ORDER BY NombreLocation DESC
LIMIT 20
'''

df = requests_sql(query, conn)
df.head()

,NomClient,NombreLocation,Ranking
0,ELEANOR HUNT,46,1
1,KARL SEAL,45,2
2,CLARA SHAW,42,3
3,MARCIA DEAN,42,3
4,TAMMY SANDERS,41,5


In [8]:
# Cas N°2 : Revenus par mois
# But : Calcule le montant total encaissé par mois et par année, trié chronologiquement.

query = '''
SELECT
    STRFTIME('%Y', payment.payment_date) AS Annee,
    STRFTIME('%m', payment.payment_date) AS Mois,
    SUM(payment.amount) AS MontantTotal
FROM payment
GROUP BY Annee, Mois
ORDER BY Annee, Mois
'''

df = requests_sql(query, conn)
df.head()

,Annee,Mois,MontantTotal
0,2005,05,4824.43
1,2005,06,9631.88
2,2005,07,28373.89
3,2005,08,24072.13
4,2006,02,514.18


In [9]:
# Cas N°3 : Classement CA par client
# But : Classe tous les clients par chiffre d’affaires total et donne leur rang (1 = plus gros CA)

query = '''
SELECT
    customer.first_name || ' ' || customer.last_name AS NomClient,
    SUM(payment.amount) AS MontantTotalClient,
    RANK() OVER (ORDER BY COALESCE(SUM(payment.amount), 0) DESC) AS Ranking
FROM customer
JOIN payment 
    ON customer.customer_id = payment.customer_id
GROUP BY customer.customer_id
ORDER BY Ranking, NomClient
'''

df = requests_sql(query, conn)
df.head()

,NomClient,MontantTotalClient,Ranking
0,KARL SEAL,221.55,1
1,ELEANOR HUNT,216.54,2
2,CLARA SHAW,195.58,3
3,MARION SNYDER,194.61,4
4,RHONDA KENNEDY,194.61,4


In [10]:
# Cas N°4 : Films jamais loués
# But : Lister tous les films présents au catalogue qui n’ont jamais été loués.

query = '''
SELECT
    film.title,
    COUNT(rental.rental_id) AS NombreLocations,
    COUNT(inventory.inventory_id) AS NombreCopies
FROM film
LEFT JOIN inventory 
    ON film.film_id = inventory.film_id
LEFT JOIN rental
    ON inventory.inventory_id = rental.inventory_id
GROUP BY film.film_id, film.title
HAVING NombreCopies > 0 AND COUNT(rental.rental_id) = 0
ORDER BY film.title
'''

df = requests_sql(query, conn)
df.head()

,title,NombreLocations,NombreCopies


In [11]:
# Cas n°5 : Top catégories par volume
# But : Pour chaque catégorie, donner le nombre total de locations et afficher les 5 catégories avec le plus de locations.

query = '''
SELECT
    category.name AS NomCategorie,
    COALESCE(COUNT(rental.rental_id), 0) AS TotalLocations
FROM category
LEFT JOIN film_category
    ON category.category_id = film_category.category_id
LEFT JOIN inventory
    ON film_category.film_id = inventory.film_id
LEFT JOIN rental
    ON inventory.inventory_id = rental.inventory_id
GROUP BY category.category_id, category.name
ORDER BY TotalLocations DESC, NomCategorie
LIMIT 5
'''

df = requests_sql(query, conn)
df.head()

,NomCategorie,TotalLocations
0,Sports,1179
1,Animation,1166
2,Action,1112
3,Sci-Fi,1101
4,Family,1096


In [12]:
# Cas n°6 : Durée moyenne de location par catégorie
# But : Pour chaque catégorie, calculer la durée moyenne entre rental_date et return_date (en jours).

query = '''
SELECT
    category.name AS NomCategorie,
    COUNT(rental.rental_id) AS NbLocations,
    ROUND(AVG(JULIANDAY(rental.return_date) - JULIANDAY(rental.rental_date)),1) AS MoyenneDureeLocation,
    ROUND(MIN(JULIANDAY(rental.return_date) - JULIANDAY(rental.rental_date)),1) AS MinimumDureeLocation,
    ROUND(MAX(JULIANDAY(rental.return_date) - JULIANDAY(rental.rental_date)),1) AS MoaximumDureeLocation
FROM category
LEFT JOIN film_category
    ON category.category_id = film_category.category_id
LEFT JOIN inventory
    ON film_category.film_id = inventory.film_id
LEFT JOIN rental
    ON inventory.inventory_id = rental.inventory_id
WHERE rental.return_date IS NOT NULL
GROUP BY category.category_id, category.name
HAVING NbLocations > 0
ORDER BY MoyenneDureeLocation DESC, NomCategorie ASC
'''

df = requests_sql(query, conn)
df.head()

,NomCategorie,NbLocations,MoyenneDureeLocation,MinimumDureeLocation,MoaximumDureeLocation
0,Games,955,5.2,0.8,9.2
1,Sports,1164,5.2,0.8,9.2
2,Comedy,932,5.1,0.8,9.2
3,Documentary,1043,5.1,0.8,9.2
4,Family,1083,5.1,0.8,9.2


In [13]:
# Cas n°7 : Top acteurs par nombre de locations
# But : Donner le top 10 des acteurs apparaissant dans des films ayant généré le plus de locations (afficher acteur + nombre de locations).

query = '''
SELECT
    actor.first_name || ' ' || actor.last_name AS NomActeur,
    COUNT(rental.rental_id) AS TotalLocations
FROM actor
JOIN film_actor
    ON actor.actor_id = film_actor.actor_id
JOIN inventory
    ON film_actor.film_id = inventory.film_id
JOIN rental
    ON inventory.inventory_id = rental.inventory_id
GROUP BY actor.actor_id, actor.first_name, actor.last_name
ORDER BY TotalLocations DESC, NomActeur ASC
LIMIT 10
'''

df = requests_sql(query, conn)
df.head()

,NomActeur,TotalLocations
0,GINA DEGENERES,753
1,MATTHEW CARREY,678
2,MARY KEITEL,674
3,ANGELA WITHERSPOON,654
4,WALTER TORN,640


In [14]:
# Cas n°8 : Taux d’activation des clients par magasin
# But : Pour chaque magasin (store), calculer la part (%) de clients actifs parmi tous ses clients.
# Afficher : store_id, nombre total de clients, nombre de clients actifs, taux d’activation (%) arrondi à 1 décimale.

query = '''
SELECT
    store.store_id AS NumeroMagasin,
    COUNT(customer.customer_id) AS ClientsTotaux,
    SUM(CASE WHEN customer.active = 1 THEN 1 ELSE 0 END) AS ClientsActifs,
    (COUNT(customer.customer_id) - SUM(CASE WHEN customer.active = 1 THEN 1 ELSE 0 END)) AS ClientsInactifs,
    ROUND(100.0 * SUM(CASE WHEN customer.active = 1 THEN 1 ELSE 0 END) / COUNT(customer.customer_id), 1) AS TauxActivation
FROM store
JOIN customer
    ON store.store_id = customer.store_id
GROUP BY store.store_id
ORDER BY TauxActivation DESC, NumeroMagasin ASC
'''

df = requests_sql(query, conn)
df.head()

,NumeroMagasin,ClientsTotaux,ClientsActifs,ClientsInactifs,TauxActivation
0,1,326,318,8,97.5
1,2,273,266,7,97.4


In [15]:
# Cas n°9 : Comparaison Mois N vs Mois N-1
# But : Pour chaque mois présent dans payment, comparer le CA à celui du mois précédent, calculer l’évolution en %.

query = '''
WITH GainsMois AS (
    SELECT
        STRFTIME('%Y-%m', payment.payment_date) AS AnneeMois,
        SUM(payment.amount) AS CaMensuel
    FROM payment
    GROUP BY AnneeMois
),
AvecPrecedent AS (
    SELECT
        AnneeMois,
        CaMensuel,
        LAG(CaMensuel) OVER (ORDER BY AnneeMois) AS CaMoisPrecedent
    FROM GainsMois
)
SELECT
    AnneeMois,
    CaMensuel,
    CaMoisPrecedent,
    CASE
        WHEN CaMoisPrecedent IS NULL OR CaMoisPrecedent = 0 THEN NULL
        ELSE ROUND(100.0 * (CaMensuel - CaMoisPrecedent) / CaMoisPrecedent, 1)
    END AS EvolPct
FROM AvecPrecedent
ORDER BY AnneeMois DESC
'''

df = requests_sql(query, conn)
df.head()

,AnneeMois,CaMensuel,CaMoisPrecedent,EvolPct
0,2006-02,514.18,24072.13,-97.9
1,2005-08,24072.13,28373.89,-15.2
2,2005-07,28373.89,9631.88,194.6
3,2005-06,9631.88,4824.43,99.6
4,2005-05,4824.43,NaN,NaN


In [16]:
# Cas n°10 : Films les plus rentables vs coût de remplacement
# But : Lister les 20 films ayant généré le plus de revenus et afficher, pour chacun :
# le revenu total
# le coût de remplacement (replacement_cost)
# le ratio revenu_total / replacement_cost arrondi à 2 décimales

query = '''
SELECT
    film.title AS TitreFilm,
    film.replacement_cost AS CoutRemplacement,
    SUM(payment.amount) AS GainsFilm,
    ROUND(1.0 * SUM(payment.amount) / film.replacement_cost, 2) AS RatioRevenuRemplacement
FROM film
JOIN inventory 
    ON film.film_id = inventory.film_id
JOIN rental
    ON inventory.inventory_id = rental.inventory_id
JOIN 
    payment
    ON rental.rental_id = payment.rental_id
GROUP BY film.film_id, film.title, film.replacement_cost
ORDER BY RatioRevenuRemplacement DESC, GainsFilm DESC
LIMIT 20
'''

df = requests_sql(query, conn)
df.head()

,TitreFilm,CoutRemplacement,GainsFilm,RatioRevenuRemplacement
0,TITANS JERK,11.99,201.71,16.82
1,MAIDEN HOME,9.99,163.76,16.39
2,VIDEOTAPE ARSENIC,10.99,178.71,16.26
3,STING PERSONAL,9.99,159.79,15.99
4,WITCHES PANIC,10.99,173.70,15.81


<h2 style="font-size:40px; font-weight:bold;">4 : Questions Entretien</h2>

Q1 : INNER JOIN vs LEFT JOIN                                                                        
R1 : INNER JOIN ou JOIN permet de récupérer uniquement les lignes qui correspondent dans les deux tables.                                    
     LEFT JOIN récupère toutes les lignes de la table de gauche et met NULL dans les colonnes de droite si pas de correspondance.                        

Q2 : GROUP BY vs HAVING                             
R2 : GROUP BY agrège les données et diminue la granularité.                               
     WHERE filtre avant agrégation, HAVING filtre après agrégation.                                 

Q3 : Gestion des NULL                                     
R3 : COALESCE remplace les valeurs nulles par une valeur par défaut (ex. 0).                                 
     IS NULL filtre les résultats contenant des NULL.                                    

Q4 : WHERE vs HAVING (pièges)                               
R4 : WHERE filtre avant agrégation,                                                                         
     HAVING filtre après.                                                

Q5 : Fenêtres analytiques                                                
R5 : ROW_NUMBER : numérote les lignes (pas de gestion des égalités).                                             
     RANK : gère les égalités mais saute des numéros.                                               
     DENSE_RANK : gère les égalités sans sauter de numéros.                                        
     LAG : valeur précédente, LEAD : valeur suivante.                                           
     Peut être partitionné par catégorie, année... et ordonné avec ORDER BY.                                           

Q6 : UNION vs UNION ALL                                        
R6 : UNION supprime les doublons (plus lent).                                             
     UNION ALL garde les doublons (plus rapide).                                      

Q7 : Validation des résultats                                         
R7 : Vérifier le nombre total de lignes avant/après filtrage.                                                
     Comparer avec une autre méthode (Excel, BI...).                                      
     Croiser avec un échantillon vérifié manuellement.                                      

Q8 : Optimisation d'une requête lente                                          
R8 : Filtrer le plus tôt possible.                                    
     Éviter SELECT *.                                         
     Garder seulement les colonnes utiles.                                        
     Supprimer sous-requêtes inutiles ou utiliser des CTE.                                                 
     Limiter les lignes pendant les tests.                                     

Q9 : Qualité des données                                               
R9 : Doublons : COUNT(DISTINCT ...) ou recherche de clés dupliquées.                                       
     Valeurs aberrantes : min/max hors plage attendue.                                             
     Dates incohérentes : return_date < rental_date.                                          
     Valeurs manquantes : IS NULL dans les colonnes clés.                              

Q10 : KPI Sakila                                          
R10 : CA mensuel/annuel vs précédent, meilleur vendeur, meilleur magasin,                              
      film ou acteur le plus loué, meilleurs clients.                                  

Q11 : Granularité                                       
R11 : Mauvaise granularité = KPI faux.                                            
      Le choix dépend de l’usage et du destinataire final.                                    

Q12 : Comparaison temporelle                                             
R12 : Utiliser LAG ou LEAD.                                
      Vérifier les mois incomplets avant comparaison.                                         

Q13 : Nettoyage texte                                   
R13 : TRIM : enlever espaces début/fin.                                        
      UPPER / LOWER : majuscules / minuscules.                                         
      REPLACE : remplacer X par Y.                                  
      SUBSTRING : extraire partie du texte.                                     

Q14 : Outliers                                  
R14 : Repérer avec AVG ± 2*STDDEV, percentiles, ou seuil défini par le client.                           

Q15 : Storytelling analytique                     
R15 : Vérifier que la baisse est réelle.                                   
      Identifier période exacte et comparer N-1 ou MoM.                             
      Segmenter (magasin, catégorie, client).                                          
      Rechercher causes externes (saison, stock, promo).                                       

Q16 : Réutilisabilité                                              
R16 : Transformer en CTE pour réutilisation interne.                                               
      Ou stocker en vue (VIEW) pour l’équipe.                                             

<h2 style="font-size:40px; font-weight:bold;">5 : Modifications de table</h2>

In [17]:
# Cas n°11 : CRUD basique (Create, Read, Update, Delete)
# But : Illustrer les opérations d’écriture/lecture SQL sur une table temporaire.

# Create : Création d'une table temporaire
conn.execute('''
CREATE TEMP TABLE tmp_acteur (
    actor_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL
)
''')

# Insert : Ajouter un acteur fictif
conn.execute('''
INSERT INTO tmp_acteur (actor_id, first_name, last_name)
VALUES (1, 'Jean', 'Dupont')
''')

# Read : Vérification de l'insertion
query = '''
SELECT * FROM tmp_acteur
'''
df = requests_sql(query, conn)
df.head()

,actor_id,first_name,last_name
0,1,Jean,Dupont


In [18]:
# Update : Modifier le nom de l’acteur
conn.execute('''
UPDATE tmp_acteur
SET last_name = 'Durand'
WHERE actor_id = 1
''')

# Read : Vérification de la modification 
df = requests_sql('SELECT * FROM tmp_acteur', conn)
df.head()

,actor_id,first_name,last_name
0,1,Jean,Durand


In [19]:
# Delete : Supprimer l’acteur
conn.execute('''
DELETE FROM tmp_acteur
WHERE actor_id = 1
''')

# Read : Vérification de la suppression
df = requests_sql('SELECT * FROM tmp_acteur', conn)
df.head()

,actor_id,first_name,last_name
